In [1]:
import re
import pandas as pd
import numpy as np
import sqlite3
import os

# change directory 
os.chdir(r'C:\Users\Rodzinka\Desktop\siatka_23')

In [2]:
# define function for rolling mean calculation
def rolling_diff(df, list_of_n):
    temp_df = pd.DataFrame()
    counter = 0
    for n in list_of_n:
        for column in df.columns[:7]:
            new_name = 'Last_' + str(n) + '_' + column
            temp_df[new_name] = df[column].rolling(window=n+1).apply(lambda x: x.iloc[-1] - x.iloc[0])
        for j in range(1, n):
            temp_df.iloc[j,counter:counter+7] = df.iloc[j,:7]
        counter = counter + 7
    temp_df["Round"] = df["Round"]
    temp_df['Year'] = df['Year']
    temp_df['Team_name'] = df['Team_name']
    return temp_df

In [3]:
# import data from db 

# define years 
years = ['2010/2011','2011/2012','2012/2013','2013/2014','2014/2015','2015/2016','2016/2017','2017/2018',
         '2018/2019','2019/2020','2020/2021','2021/2022', '2022/2023', '2023/2024']

# change team names to ones used in other notebooks 
names = ['Aluron', 'AZS Częstochowa', 'Asseco', 'BBTS', 'Bydgoszcz', 'Kielce', 'Szczecin', 'Czarni', 'Lubin', 
         'Indykpol', 'GKS', 'Węgiel', 'Będzin', 'Wieluń',
         'Lublin', 'Skra', 'Warsza', 'Nysa', 'Suwałki', 'ZAKSA', 'Gdańsk', 'Hemarpol',
         'www']

# create empty table for storing results
tab = pd.DataFrame()

# loop over all seasons
for year in years:
    # read database, change path and database name if needed
    year_ = year.split('/')[0]  
    conn = sqlite3.connect('data/database_rounds_{}.db'.format(year_))
    cursor = conn.cursor()
    result = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

    # find table names - match data
    table_names = sorted(list(zip(*result))[0])

    # create first table
    test_name = table_names[0]
    data = pd.read_sql("SELECT * FROM '{}'".format(test_name), conn)
    data['Round'] = test_name.split('_')[1]
    
    # append all match stats to one table
    for name in table_names[1:]:
        data_i = pd.read_sql("SELECT * FROM '{}'".format(name), conn)
        data_i['Round']= name.split('_')[1]
        # concatenate created table with whole data
        data = pd.concat([data, data_i])
    
    # change team name
    for name in names:
        data.iloc[:,1] = data.iloc[:,1].apply(lambda x: name if not pd.isnull(x) and name.lower() in x.lower() else x)
    data

    # add year
    data['Year'] = year
    
    # concatenate results from each year
    tab = pd.concat([tab, data])
    
tab

,"('Unnamed: 0_level_0', 'Unnamed: 0_level_1')","('Drużyna', 'Drużyna')","('Punkty', 'Punkty')","('Mecze', 'Rozegr.')","('Mecze', 'Wygr.')","('Mecze', 'Przegr.')","('Sety', 'Wygr.')","('Sety', 'Przegr.')","('Punkty', 'Wygr.')","('Punkty', 'Przegr.')","('Stosunek', 'Sety')","('Stosunek', 'Punkty')","('Wynik', '3-0')","('Wynik', '3-1')","('Wynik', '3-2')","('Wynik', '2-3')","('Wynik', '1-3')","('Wynik', '0-3')",Round,Year
0,1,Skra,3,1,1,0,3,0,75,54,-,1.3889,1,0,0,0,0,0,1,2010/2011
1,2,ZAKSA,3,1,1,0,3,0,75,54,-,1.3889,1,0,0,0,0,0,1,2010/2011
2,3,Warsza,3,1,1,0,3,1,94,93,3.0000,1.0108,0,1,0,0,0,0,1,2010/2011
3,4,Asseco,2,1,1,0,3,2,114,107,1.5000,1.0654,0,0,1,0,0,0,1,2010/2011
4,5,Bydgoszcz,2,1,1,0,3,2,103,101,1.5000,1.0198,0,0,1,0,0,0,1,2010/2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,12,Lubin,8,9,2,7,10,21,663,722,04762,9183,2,0,0,2,0,5,9,2023/2024
12,13,Skra,7,8,3,5,13,20,734,776,06500,9459,0,1,2,0,4,1,9,2023/2024
13,14,Hemarpol,7,9,2,7,11,23,752,821,04783,9160,0,2,0,1,3,3,9,2023/2024
14,15,GKS,3,9,1,8,8,26,706,815,03077,8663,0,0,1,1,3,4,9,2023/2024


In [4]:
# change data type in order to perform calculations
tab['Round'] = pd.to_numeric(tab['Round'])

# choose columns and change their names
tab = tab.iloc[:, [2, 4, 5, 6, 7, 8, 9, 18, 19, 1]]
tab.columns = ['Points', 'Matches_won', 'Matches_lost', 'Sets_won', 'Sets_lost', 'Points_won', 'Points_lost',
                 'Round','Year', 'Team_name']

# round data
tab = tab.round(2)


In [6]:
tab['Round'] = tab['Round'] + 1
tab

,Points,Matches_won,Matches_lost,Sets_won,Sets_lost,Points_won,Points_lost,Round,Year,Team_name
0,3,1,0,3,0,75,54,2,2010/2011,Skra
1,3,1,0,3,0,75,54,2,2010/2011,ZAKSA
2,3,1,0,3,1,94,93,2,2010/2011,Warsza
3,2,1,0,3,2,114,107,2,2010/2011,Asseco
4,2,1,0,3,2,103,101,2,2010/2011,Bydgoszcz
...,...,...,...,...,...,...,...,...,...,...
11,8,2,7,10,21,663,722,10,2023/2024,Lubin
12,7,3,5,13,20,734,776,10,2023/2024,Skra
13,7,2,7,11,23,752,821,10,2023/2024,Hemarpol
14,3,1,8,8,26,706,815,10,2023/2024,GKS


In [7]:
# save results to db
conn21 = sqlite3.connect('data/database_rounds_prepared.db')
tab.to_sql("Rounds_prep", conn21, if_exists='replace', index=False)   

4588

In [12]:
# create empty table for storing the results
final = pd.DataFrame()

# define years 
years = ['2010/2011','2011/2012','2012/2013','2013/2014','2014/2015','2015/2016','2016/2017','2017/2018',
         '2018/2019','2019/2020','2020/2021','2021/2022', '2022/2023', '2023/2024']

# define team names
names = ['Aluron', 'AZS Częstochowa', 'Asseco', 'BBTS', 'Bydgoszcz', 'Kielce', 'Szczecin', 'Czarni', 'Lubin', 
         'Indykpol', 'GKS', 'Węgiel', 'Będzin', 'Wieluń',
         'Lublin', 'Skra', 'Warsza', 'Nysa', 'Suwałki', 'ZAKSA', 'Gdańsk', 'Hemarpol' 
         'www']

for year in years:
    for name in names:

        temp_ = tab[tab['Team_name']==name]
        temp_ = temp_[temp_['Year']==year]
        temp_ = temp_.sort_values(by=['Round']).reset_index(drop=True)
        
        if temp_.empty:
            print('DataFrame is empty!')
            
        else:
            temp_.iloc[-1] = temp_.iloc[0]  # adding a row - there is no reliable data for macthes played during the first round of the season 
            temp_.iloc[-1, 7] = 0
            temp_ = temp_.sort_values(by=['Round'])
            temp_.iloc[0, :7] = 0
            temp_ = temp_.reset_index(drop=True) 
            
            # calculate rolling differences
            temp_ = rolling_diff(temp_, [1, 3, 5])
            
            # increase round number in order to prevent data leakage
            temp_['Round'] += 1

            final = pd.concat([final, temp_])
final

DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!
DataFrame is empty!


,Last_1_Points,Last_1_Matches_won,Last_1_Matches_lost,Last_1_Sets_won,Last_1_Sets_lost,Last_1_Points_won,Last_1_Points_lost,Last_3_Points,Last_3_Matches_won,Last_3_Matches_lost,...,Last_5_Points,Last_5_Matches_won,Last_5_Matches_lost,Last_5_Sets_won,Last_5_Sets_lost,Last_5_Points_won,Last_5_Points_lost,Round,Year,Team_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2010/2011,AZS Częstochowa
1,1.0,0.0,1.0,2.0,3.0,101.0,103.0,1.0,0.0,1.0,...,1.0,0.0,1.0,2.0,3.0,101.0,103.0,2,2010/2011,AZS Częstochowa
2,1.0,0.0,1.0,2.0,3.0,106.0,107.0,2.0,0.0,2.0,...,2.0,0.0,2.0,4.0,6.0,207.0,210.0,3,2010/2011,AZS Częstochowa
3,3.0,1.0,0.0,3.0,0.0,75.0,53.0,5.0,1.0,2.0,...,5.0,1.0,2.0,7.0,6.0,282.0,263.0,4,2010/2011,AZS Częstochowa
4,3.0,1.0,0.0,3.0,1.0,97.0,91.0,7.0,2.0,1.0,...,8.0,2.0,2.0,10.0,7.0,379.0,354.0,5,2010/2011,AZS Częstochowa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1.0,0.0,1.0,2.0,3.0,99.0,104.0,7.0,2.0,1.0,...,13.0,4.0,1.0,14.0,4.0,436.0,391.0,6,2023/2024,Gdańsk
6,3.0,1.0,0.0,3.0,1.0,99.0,91.0,7.0,2.0,1.0,...,13.0,4.0,1.0,14.0,5.0,460.0,422.0,7,2023/2024,Gdańsk
7,0.0,0.0,1.0,0.0,3.0,66.0,76.0,4.0,1.0,2.0,...,10.0,3.0,2.0,11.0,7.0,415.0,394.0,8,2023/2024,Gdańsk
8,3.0,1.0,0.0,3.0,0.0,86.0,71.0,6.0,2.0,1.0,...,10.0,3.0,2.0,11.0,7.0,426.0,401.0,9,2023/2024,Gdańsk


In [13]:
# save created data as sql
conn3 = sqlite3.connect('data/database_rounds_averages_all_teams.db')
final.to_sql("Rounds", conn3, if_exists='replace', index=False)   

4538

In [14]:
# close connection
cursor.close() 
conn.close()
conn3.close()